- А точно нужно сортировать по вероятности?

Это повысит вероятность покупки.

- Какую метрику использовать?

Так как хотим повысить выручку за счет рекоммендаций, то лучше использовать метрику money_precision.

- В какое время отправляем рассылку?

Отправка рассылки в вечернее время для локации пользователя повысит вероятность того, что она будет прочитана.

- Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?

Добавлять товары, купленные другими пользователями, но еще не приобретенными нашим, если существенная часть (например, $> 70$%) приобретенных целевым пользователем товаров была приобретена ими.

- Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?

Можно выбирать товары из разных категорий и об этом не придется задумываться.

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.sparse import csr_matrix
from sklearn.model_selection import ParameterGrid
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

In [12]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [13]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,25671,26081,26093,26190,26355,26426,26540,26601,26636,26691,...,17328742,17329473,17329749,17330255,17330511,17381856,17382205,17383227,17827644,17829232
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
def precision_at_k(result_recomendation, actual_recomendation, k=5):
    top_k = result_recomendation[:k]
    num_correct = len(set(top_k).intersection(actual_recomendation))
    precision = num_correct / k
    return precision

In [15]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [16]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [17]:
user_item_matrix = bm25_weight(user_item_matrix.T).T 

In [18]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [26]:
params_grid = {
    'factors': [50, 100, 150, 200],
    'regularization': [0.001, 0.01, 0.1],
    'iterations': [10, 15, 20, 25],
    'calculate_training_loss': [True],
    'num_threads': [4]
    }

In [27]:
%%time

best_precision = 0
best_params = {}
for params_grid_dict in ParameterGrid(params_grid):
    model = AlternatingLeastSquares(**params_grid_dict)
    model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)
    result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
    precision = result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()
    if precision > best_precision:
        best_precision = precision
        best_params = params_grid_dict
print(f'precision_at_5 = {best_precision}, best params = {best_params}')

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

precision_at_5 = 0.2457394711067581, best params = {'calculate_training_loss': True, 'factors': 200, 'iterations': 25, 'num_threads': 4, 'regularization': 0.001}
CPU times: user 14h 52min 24s, sys: 1d 6h 40min 57s, total: 1d 21h 33min 21s
Wall time: 3h 59min 32s
